In [1]:
'''
Energy model to compare with measurement
Default size unit: mm
Author: Yitian Shao
Created on 2022.01.15
'''
%matplotlib notebook 
%matplotlib notebook 

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 14})

from PouchLib import *

try:
    EPSILON0 = getEpsilon0() # (Farad/m) Free-space permittivity
except:
    EPSILON0 = 0.0000000000088541878128

c shared library imported successfully!


In [2]:
'''
General Functions
'''

def aPlot(figName, is3D = False):
    ax = []
    
    fig1 = plt.figure(figsize = (10,4))
    fig1.suptitle(figName, fontsize=16)
    if(is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax, fig1

def stateEnergy(U, zipTriNum, zipRectNum, deltaTriCapa, deltaRectCapa, deltaTriVol, deltaRectVol):
    # 'U' - Voltage applied, '*Capa' - Capacitance, '*Vol' - Volume of fluid
    
    deltaC = zipTriNum * deltaTriCapa+ zipRectNum * deltaRectCapa
    deltaEs = 0.5 * U*U * deltaC # Unit: Joule
    
    deltaVol = (zipTriNum * deltaTriVol + zipRectNum * deltaRectVol) * 1e-9 # (Unit: mm3 to m3) Volume of the entire pouch 
    
    avgPressure = deltaEs / deltaVol # (Pa)
    
    print("avgP = %.1fkPa, deltaV = %.1f mL" % (avgPressure*1e-3, deltaVol*1e6))
    
    return avgPressure, (2*deltaEs), deltaVol # 'Avg.' pressure (Pa) and Total electrical energy input (Joule) to the actuator


In [3]:
''' Import tables and data '''
condiTable = pd.read_csv("./data/ConditionTable.csv") # Table of measurement conditions
print(condiTable)

materProperty = pd.read_csv("./data/MaterialProperty.csv") # Table of material properties
print(materProperty)

measureData = pd.read_csv("./data/Measurements.csv") # Data from physical measurements
print(measureData.head(1))

stateLayout = pd.read_csv("./data/StateLayout.csv") # Design layout and state 
print(stateLayout)

md = pd.read_csv("./data/ValidArcLength.csv") # Models of various parameters with valid arc length
print(md.head(1))

             CondiName  Shell  ls_um     Fluid  Infill_mL  DashLength_mm  \
0  MLSi20HV7kV10mL.5Hz  Mylar     15  Silicone         10             20   
1   MLSi20HV7kV10mL1Hz  Mylar     15  Silicone         10             20   
2   MLSi20HV7kV10mL2Hz  Mylar     15  Silicone         10             20   
3   MLSi20HV7kV10mL4Hz  Mylar     15  Silicone         10             20   
4  MLSi20HV7kV10mL10Hz  Mylar     15  Silicone         10             20   
5  MLSi20HV7kV10mL20Hz  Mylar     15  Silicone         10             20   

   FlatDashDist_mm  DashSpace_mm     Layout  triNum  rectNum  TubeLength_mm  \
0               10          8.66  T5T4T3T2T      45       18            150   
1               10          8.66  T5T4T3T2T      45       18            150   
2               10          8.66  T5T4T3T2T      45       18            150   
3               10          8.66  T5T4T3T2T      45       18            150   
4               10          8.66  T5T4T3T2T      45       18            

In [4]:
''' Compute change of volume and capacitance under different state for each condition '''
intStepSize = 0.0001 # (mm) Step size for performing integral computation

res = []
for i in condiTable.index:
    layoutLabel = condiTable.loc[i,'Layout']
    if layoutLabel == '16C?': # To check what is the exact design of pouch "16C" !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        layoutLabel = 'T5T4T3T2T1TR'
    
    print("\n%s layout: %s" % (condiTable.loc[i,'CondiName'], layoutLabel))
    
    ''' Identify design layout of the model by matching dash Space, dash Length, and Infill Volume '''
    temp = md[(md['dashSpace'] == condiTable.loc[i,'DashSpace_mm']) & (md['dashLength'] == condiTable.loc[i,'DashLength_mm'])]   
    mod0 = temp.iloc[(temp[layoutLabel] - 1000*condiTable.loc[i,'Infill_mL']).abs().argsort()[:1]] # Unit: mL converted to mm3
#     print(mod0.head())
    
    
    ''' Loopup material properties '''
    shellThickness = condiTable.loc[i,'ls_um'] / 1000 # Unit: um converted to mm
    shellMaterial = condiTable.loc[i,'Shell']
    fluidMaterial = condiTable.loc[i,'Fluid']
    print("shell: %s, fluid: %s" % (shellMaterial, fluidMaterial))
    
    if shellMaterial == 'PETL0Ws': # Dielectric constant for PETL0Ws is unclear!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        epsilon_s = 3.0 # This number for PETL0Ws need to be checked !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    else:
        epsilon_s = materProperty.loc[materProperty['Material']==shellMaterial,'DieConstant'].iloc[0]
    
    epsilon_f = materProperty.loc[materProperty['Material']==fluidMaterial,'DieConstant'].iloc[0]

    
    ''' Compuate capacitance of inactive and zipped state'''
    # Triangle Pouch ---------------------------------------------------------------------
    triPouch0 = TrianglePouch(mod0['R'].iloc[0], mod0['dashSpace'].iloc[0], mod0['dashHalfDist'].iloc[0])
    #triPouch0.printInfo()

    triCapa = triPouch0.getCapacitance(shellThickness, epsilon_s, epsilon_f, intStepSize) # This varies by materials
    print("Capacitance of inactive triangle pouch = %.3f Picofarad" % (triCapa*1e12))

    # Rectangle Pouch --------------------------------------------------------------------
    rectPouch0 = RectanglePouch(triPouch0.r, mod0['dashLength'].iloc[0], triPouch0.m, 2*triPouch0.m/triPouch0.dy)  

    rectCapa = rectPouch0.getCapacitance(shellThickness, epsilon_s, epsilon_f, intStepSize) # This varies by materials
    print("Capacitance of inactive rectangle pouch = %.3f Picofarad" % (rectCapa*1e12))

    triArea = mod0['dashSpace'].iloc[0] * mod0['dashHalfDist'].iloc[0] # (mm2) Top-view area (Not surface area)
    rectArea = 2 * mod0['dashHalfDist'].iloc[0] * mod0['dashLength'].iloc[0] # (mm2) Top-view area (Not surface area)
    ''' Compuate capacitance of zipped cells '''
    zippedTriCapa = triArea * 1e-6 * EPSILON0 * epsilon_s / (2 * shellThickness * 1e-3) # Unit: Farad
    zippedRectCapa = rectArea  * 1e-6 * EPSILON0 * epsilon_s / (2 * shellThickness * 1e-3) # Unit: Farad
    print("Capacitance of fully zipped triangle cell is %f and rectangle cell is %f Picofarad" % 
          (zippedTriCapa*1e12, zippedRectCapa*1e12))
    
    deltaTriCapa = zippedTriCapa - triCapa # Change of capacitance when a triangle cell is fully zipped
    deltaRectCapa = zippedRectCapa - rectCapa # Change of capacitance when a rectangle cell is fully zipped
    
    
    ''' Partially zipped with fluid trapped in cells '''
    diamFullExpand = condiTable.loc[i,'FlatDashDist_mm'] * 2 / np.pi # Diameter of cell fully expanded by fluid
    print("Rectangle cell fully expanded has a diameter = %.3f mm" % diamFullExpand)

    
    ''' Zipping state: Fully zipped, 1 fluid path expanded, 2 fluid path expanded '''
    ind = (stateLayout['Layout']==layoutLabel)
    
    triNum = stateLayout.loc[ind,'triNum'].iloc[0] 
    rectNum = stateLayout.loc[ind,'rectNum'].iloc[0] 
    
    ex1TriNum = triNum + stateLayout.loc[ind,'triNum1Expan'].iloc[0]
    ex1RectNum = rectNum + stateLayout.loc[ind,'rectNum1Expan'].iloc[0]
    
    ex2TriNum = triNum + stateLayout.loc[ind,'triNum2Expan'].iloc[0]
    ex2RectNum = rectNum + stateLayout.loc[ind,'rectNum2Expan'].iloc[0]
    print("(Full-1Expan-2Expan) Tri: %d-%d-%d, Rect: %d-%d-%d" % (triNum,ex1TriNum,ex2TriNum,rectNum,ex1RectNum,ex2RectNum))
    
    appliedVoltage = condiTable.loc[i,'Voltage_V']
    print("Voltage applied = %d V" % appliedVoltage)
    PFull, EsFull, dVFull = stateEnergy(appliedVoltage, triNum, rectNum, deltaTriCapa, deltaRectCapa, 
                                mod0['triVol'].iloc[0],  mod0['rectVol'].iloc[0])
    P1Expan, Es1Expan, dV1Expan = stateEnergy(appliedVoltage, ex1TriNum, ex1RectNum, deltaTriCapa, deltaRectCapa, 
                                mod0['triVol'].iloc[0],  mod0['rectVol'].iloc[0])
    P2Expan, Es2Expan, dV2Expan = stateEnergy(appliedVoltage, ex2TriNum, ex2RectNum, deltaTriCapa, deltaRectCapa, 
                                mod0['triVol'].iloc[0],  mod0['rectVol'].iloc[0])
    P1Cell, Es1Cell, dV1Cell = stateEnergy(appliedVoltage, 1, 1, deltaTriCapa, deltaRectCapa, 
                                mod0['triVol'].iloc[0],  mod0['rectVol'].iloc[0])
      
#     if False: # Visualization
#         fig1 = plt.figure(figsize = (6,6)) 
#         ax = fig1.add_subplot(111, projection='3d')
#         ax.set_xlabel('X (mm)')
#         ax.set_ylabel('Y (mm)')
#         ax.set_zlabel('Z (mm)')
#         triPouch0.displayPouch(ax, dispAdditive = True) 
#         rectPouch0.transformPouch(triPouch0.triangleBottomX, 0, 0)
#         rectPouch0.displayPouch(ax, dispAdditive = True) 
#         ax.set_xlim3d([triPouch0.triangleTopX, triPouch0.triangleTopX+40]); 
#         ax.set_ylim3d([-20, 20]); 
#         ax.set_zlim3d([-20, 20]); 
#         ax.set_box_aspect([1.0, 1.0, 1.0])
#         plt.show()


    ''' Factor of pressure loss in tube (Pressure loss per unit of volumetric flow rate) '''
    dynamicViscosity = (materProperty.loc[materProperty['Material']==fluidMaterial,'Density_kg/m3'].iloc[0] * 
                          materProperty.loc[materProperty['Material']==fluidMaterial,'Viscosity_mm2/s'].iloc[0] * 1e-6) # mm2/s to m2/s 
    
    tubeInnerRadius = 0.5 * condiTable.loc[i,'TubeInnerDia_mm'] * 1e-3 # mm to m
    
    lossFactor = 8 * dynamicViscosity * condiTable.loc[i,'TubeLength_mm'] * 1e-3 / (np.pi * (tubeInnerRadius ** 4))

    
    print("dynamic viscosity = %.6f kg/m⋅s, tube inner radius = %.4f m, lossFactor = %.3f" % 
          (dynamicViscosity,tubeInnerRadius,lossFactor))
    
    res.append([condiTable.loc[i,'CondiName'], PFull, P1Expan, P2Expan, P1Cell, EsFull, Es1Expan, Es2Expan, Es1Cell,
               dVFull, dV1Expan, dV2Expan, dV1Cell, lossFactor])

'''
P: Time-averaged pressure 
Es: Change of energy
dV: Change of fluid volme
'''
res = pd.DataFrame(res, columns=['CondiName','PFull_Pa','P1Expan_Pa','P2Expan_Pa','P1Cell_Pa',
                                 'EsFull_J','Es1Expan_J','Es2Expan_J','Es1Cell_J',
                                'dVFull_m3','dV1Expan_m3','dV2Expan_m3','dV1Cell_m3','lossFactor'])
res.to_csv("temp_result.csv", index=False)
res.tail()


MLSi20HV7kV10mL.5Hz layout: T5T4T3T2T
shell: Mylar, fluid: Silicone
Capacitance of inactive triangle pouch = 0.247 Picofarad
Capacitance of inactive rectangle pouch = 3.221 Picofarad
Capacitance of fully zipped triangle cell is 36.805088 and rectangle cell is 170.000406 Picofarad
Rectangle cell fully expanded has a diameter = 6.366 mm
(Full-1Expan-2Expan) Tri: 45-35-26, Rect: 18-14-10
Voltage applied = 7000 V
avgP = 14.7kPa, deltaV = 7.7 mL
avgP = 14.7kPa, deltaV = 6.0 mL
avgP = 14.7kPa, deltaV = 4.4 mL
avgP = 16.0kPa, deltaV = 0.3 mL
dynamic viscosity = 0.004600 kg/m⋅s, tube inner radius = 0.0020 m, lossFactor = 109816910.733

MLSi20HV7kV10mL1Hz layout: T5T4T3T2T
shell: Mylar, fluid: Silicone
Capacitance of inactive triangle pouch = 0.247 Picofarad
Capacitance of inactive rectangle pouch = 3.221 Picofarad
Capacitance of fully zipped triangle cell is 36.805088 and rectangle cell is 170.000406 Picofarad
Rectangle cell fully expanded has a diameter = 6.366 mm
(Full-1Expan-2Expan) Tri: 4

,CondiName,PFull_Pa,P1Expan_Pa,P2Expan_Pa,P1Cell_Pa,EsFull_J,Es1Expan_J,Es2Expan_J,Es1Cell_J,dVFull_m3,dV1Expan_m3,dV2Expan_m3,dV1Cell_m3,lossFactor
1,MLSi20HV7kV10mL1Hz,14718.181856,14718.181856,14656.266348,16025.829787,0.22771,0.177108,0.128297,0.009964,0.000008,0.000006,0.000004,3.108591e-07,1.098169e+08
2,MLSi20HV7kV10mL2Hz,14718.181856,14718.181856,14656.266348,16025.829787,0.22771,0.177108,0.128297,0.009964,0.000008,0.000006,0.000004,3.108591e-07,1.098169e+08
3,MLSi20HV7kV10mL4Hz,14718.181856,14718.181856,14656.266348,16025.829787,0.22771,0.177108,0.128297,0.009964,0.000008,0.000006,0.000004,3.108591e-07,1.098169e+08
4,MLSi20HV7kV10mL10Hz,14718.181856,14718.181856,14656.266348,16025.829787,0.22771,0.177108,0.128297,0.009964,0.000008,0.000006,0.000004,3.108591e-07,1.098169e+08
5,MLSi20HV7kV10mL20Hz,14718.181856,14718.181856,14656.266348,16025.829787,0.22771,0.177108,0.128297,0.009964,0.000008,0.000006,0.000004,3.108591e-07,1.098169e+08


In [5]:
# selected = '6mLBOPP12' # A single test case
# intStepSize = 0.0001 # (mm) Step size for performing integral computation

# ''' Get base design parameter from the condition table '''
# dashLength = condiTable.loc[condiTable['CondiName']==selected,'DashLength_mm'].iloc[0]
# dashSpace = condiTable.loc[condiTable['CondiName']==selected,'DashSpace_mm'].iloc[0]
# infillVol = condiTable.loc[condiTable['CondiName']==selected,'Infill_mL'].iloc[0] * 1000 # Unit: mL converted to mm3

# ''' Find the model using the base design parameters '''
# temp = pouch6Col[(pouch6Col['dashLength'] == dashLength) & (pouch6Col['dashSpace'] == dashSpace)]
# mod0 = temp.iloc[(temp['totalVol']- infillVol).abs().argsort()[:1]]
# print(mod0.head())

# ''' Ger material properties from the file '''
# shellThickness = condiTable.loc[condiTable['CondiName']==selected,'ls_um'].iloc[0] / 1000 # Unit: um converted to mm
# shellMaterial = condiTable.loc[condiTable['CondiName']==selected,'Shell'].iloc[0]
# fluidMaterial = condiTable.loc[condiTable['CondiName']==selected,'Fluid'].iloc[0]
# print("shell: %s, fluid: %s" % (shellMaterial, fluidMaterial))
# epsilon_s = materProperty.loc[materProperty['Material']==shellMaterial,'DieConstant'].iloc[0]
# epsilon_f = materProperty.loc[materProperty['Material']==fluidMaterial,'DieConstant'].iloc[0]

# ''' Compute energy step 1: Compuate capacitance of inactive state'''
# # Triangle Pouch ---------------------------------------------------------------------
# triPouch0 = TrianglePouch(mod0['R'].iloc[0], mod0['dashSpace'].iloc[0], mod0['dashHalfDist'].iloc[0])
# triPouch0.printInfo()

# triCapa = triPouch0.getCapacitance(shellThickness, epsilon_s, epsilon_f, intStepSize) # This varies by materials
# print("Capacitance of inactive triangle pouch = %.3f Picofarad" % (triCapa*1e12))

# # Rectangle Pouch --------------------------------------------------------------------
# rectPouch0 = RectanglePouch(triPouch0.r, mod0['dashLength'].iloc[0], triPouch0.m, 2*triPouch0.m/triPouch0.dy)  
# rectPouch0.transformPouch(triPouch0.triangleBottomX, 0, 0)

# rectCapa = rectPouch0.getCapacitance(shellThickness, epsilon_s, epsilon_f, intStepSize) # This varies by materials
# print("Capacitance of inactive rectangle pouch = %.3f Picofarad" % (rectCapa*1e12))

# if False:
#     fig1 = plt.figure(figsize = (6,6)) # Visualization -----------------------------------
#     ax = fig1.add_subplot(111, projection='3d')
#     ax.set_xlabel('X (mm)')
#     ax.set_ylabel('Y (mm)')
#     ax.set_zlabel('Z (mm)')
#     triPouch0.displayPouch(ax, dispAdditive = True) 
#     rectPouch0.displayPouch(ax, dispAdditive = True) 
#     ax.set_xlim3d([triPouch0.triangleTopX, triPouch0.triangleTopX+40]); 
#     ax.set_ylim3d([-20, 20]); 
#     ax.set_zlim3d([-20, 20]); 
#     ax.set_box_aspect([1.0, 1.0, 1.0])
#     plt.show()


# ''' Compute energy step 2: Compuate capacitance of zipped state '''
# zippedTriCapa = mod0['triArea'].iloc[0] * 1e-6 * EPSILON0 * epsilon_s / (2 * shellThickness * 1e-3)
# zippedRectCapa = mod0['rectArea'].iloc[0]  * 1e-6 * EPSILON0 * epsilon_s / (2 * shellThickness * 1e-3)
# print("Capacitance of fully zipped triangle cell is %f and rectangle cell is %f Picofarad" % 
#       (zippedTriCapa*1e12, zippedRectCapa*1e12))

# deltaTriCapa = zippedTriCapa - twoState['triCapa'].iloc[0] # Change of capacitance when fully zipped
# deltaRectCapa = zippedRectCapa - twoState['rectCapa'].iloc[0] # Change of capacitance when fully zipped


In [6]:
''' Partially zipped with fluid trapped in cells '''
if False:
    diameterFullyExpanded = data['triArc'].iloc[0] * 2 / np.pi
    print("Cell fully expanded with fluid has a diameter = %.3f mm" % diameterFullyExpanded)

    data['oneCellExpand'] = data['triArc'] * (np.amax(pouchStructure)) + diameterFullyExpanded

    data.head()
    # data.to_csv("ToCompareLength.csv")

In [7]:
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(6,3)
# fig1.suptitle("Volume Check")
# ax1.set_xlabel('R (mm)')

# ax1.plot(data['R'], data['totalVol']/1000, color='tab:red')
# ax1.set_ylabel('Total Volume (mL)', color='tab:red')
# ax1.tick_params(axis='y', labelcolor='tab:red')

# ax2 = ax1.twinx() 
# ax2.plot(data['R'], data['totalCapa'], color='tab:blue')
# ax2.set_ylabel('Total Capacitance (Farad)', color='tab:blue')
# ax2.tick_params(axis='y', labelcolor='tab:blue')

# fig1.tight_layout() 


In [8]:
# selectCondition = 'dashSpace==8.66'
# twoState = pd.concat((data.query(selectCondition).iloc[[0]], data.query(selectCondition).iloc[[-1]]), axis=0) 
# twoState.head()


# ''' Zipped state '''
# zippedTriCapa = twoState['triArea'].iloc[0] * 1e-6 * EPSILON0 * epsilon_s / (2 * shellThickness * 1e-3)
# zippedRectCapa = twoState['rectArea'].iloc[0]  * 1e-6 * EPSILON0 * epsilon_s / (2 * shellThickness * 1e-3)
# print("Capacitance of fully zipped triangle cell is %f and rectangle cell is %f Picofarad" % (zippedTriCapa*1e12, zippedRectCapa*1e12))


# Wps = 1 # (Joule) Work done by power supply
# U = 6900 # (V) Driven voltage


# deltaC = triNum * deltaTriCapa+ rectNum * deltaRectCapa # (Farad) Increased capacitance
# deltaEs = 0.5 * U*U * deltaC
# print("Change of electrostatic potential energy (Joule) = ", deltaEs)

# deltaV = (twoState['totalVol'].iloc[0]) * 1e-9 # (m3) Decreased volume of fluid <- twoState['totalVol'].iloc[1]
# print("Change of fluid volume (m3) = ", deltaV)

# ######### avgP = (Wps - deltaEs)/deltaV # This is inaccurate since power disspation is large before supplied to the actuator
# avgP = deltaEs/deltaV # (2*deltaEs-deltaEs)
# print("Average pressure P = %f kPa" % (avgP*1e-3))

# ''' Partially zipped (reality) '''
# oneExpandTriNum = 2*(len(pouchStructure)-1)
# oneExpandRectNum = len(pouchStructure)
# partZipTriNum = triNum - oneExpandTriNum
# partZipRectNum = rectNum - oneExpandRectNum
# print("Partially zipped pouch (one expand per column) contains totally %d rectangle and %d triangle" % (partZipRectNum, partZipTriNum))

# deltaC2 = partZipTriNum * deltaTriCapa+ partZipRectNum * deltaRectCapa # (Farad) Increased capacitance when not fully zipped
# deltaEs2 = 0.5 * U*U * deltaC2
# print("(Not fully zipped) Change of electrostatic potential energy (Joule) = ", deltaEs2)

# oneExpandRectVol = (data['triArc'].iloc[0] * data['triArc'].iloc[0] / np.pi) * data['dashLength'].iloc[0] # Unit: mm3
# #deltaV2 = deltaV - (oneExpandTriNum*twoState['triVol'].iloc[0] + oneExpandRectNum*twoState['rectVol'].iloc[0]) * 1e-9 # Unit: m3
# deltaV2 = deltaV - (oneExpandTriNum*twoState['triVol'].iloc[0] + oneExpandRectNum*oneExpandRectVol) * 1e-9 # Unit: m3
# print("(Not fully zipped) Change of fluid volume (m3) = ", deltaV2)

# avgP2 = deltaEs2/deltaV2 # (2*deltaEs-deltaEs)
# print("(Not fully zipped) Average pressure P = %f kPa" % (avgP2*1e-3))